In [41]:
import json
import openai
import pickle


model_id = "gpt-4"
API_key = ""
openai.api_key=API_key

path_to_data = "/submission_X"
f = open(path_to_data)
data = json.load(f)
response_to_eval = {}

with open("response_validation_turns", 'r') as f:
    data_turns = f.readlines()

selected_turns = [elem.strip() for elem in data_turns]

for turn in data["turns"]:
    for response in turn["responses"]:
        turn_id = turn["turn_id"]
        if turn_id in selected_turns:
            if (response["rank"] == 1) or (response["rank"] == "1"):
                response_to_eval[turn_id] = response["text"]

In [43]:
prompts_per_turn = {}

prompt_naturallness = """
I will give you a text, you should evaluate the naturalness of the given text. You should say to what grade this text is natural.
Text: {answer}
Score 4 means a very natural and human-like text and 0 means a completely not natural text. Please only select an integer number between 0-4 to indicate the naturalness of the text.
"""
for turn_id in response_to_eval:
    prompts_per_turn[turn_id] = prompt_naturallness.format(answer = response_to_eval[turn_id])



In [45]:
def chatgpt_conversation(conversation_Log):
  response = openai.ChatCompletion.create(
      model = model_id,
      messages = conversation_Log
  )
  conversation_Log.append({'role':response.choices[0].message.role ,'content':response.choices[0].message.content.strip()})
  return conversation_Log

def run_one_sample(init_prompt):

    conversations = []
    conversations.append({'role': 'user', 'content': init_prompt})
    conversations = chatgpt_conversation(conversations)
    answer_r_prime = conversations[-1]['content'].strip()
    print(init_prompt)
    print("********************")
    print(answer_r_prime)
    print("**************************************")
    return answer_r_prime


In [ ]:

scores = {}
counter = 0

for turn_id in prompts_per_turn:
    counter +=1
    if counter>=0:
        X = run_one_sample(prompts_per_turn[turn_id])
        scores[turn_id] = X
        print(X)

In [48]:
len(scores)

73

In [49]:
lines = []

for turn_id in scores:
    line = turn_id + '\tsubmission-X\t' +scores[turn_id] +'\n'
    lines.append(line)

with open('/naturallness_judgement-gpt4-submission-X.txt', 'w') as f:
    f.writelines(lines)

In [12]:
import numpy as np

print(np.mean([int(score) for score in scores]))

3.3424657534246576
